*Proyecto de Análisis de Datos Cinematográficos*

Este proyecto tiene como objetivo realizar un análisis exhaustivo de datos cinematográficos utilizando la base de datos netflix.db y el archivo CSV oscar.csv. El conjunto de datos resultante ofrece información detallada sobre películas, series, directores, premiaciones, puntuaciones, ratings, duración, entre otros aspectos relevantes.

Para poder desarrollar el proyecto, fue necesario crear la base de datos a ser utilizada. Para eso usamos el archivo netflix.db y oscar.csv. Pasamos a importar las librerias que vamos a utilizar para la creación

In [1]:
import sqlite3 as sq3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from IPython.display import display  # para mostrar display con el estilo DF de Jupiter Notebook

Crear las funciones para conexión y desconexión a la base de datos

In [2]:
# CONEXIÓN
def get_connection():
    conn = sq3.connect('netflix.db')
    cur = conn.cursor()
    return conn, cur

# DESCONEXIÓN
def close_connection(conn, cur):
    cur.close()
    conn.close()

In [5]:
# Conectar a la base de datos
conn, cur = get_connection()

# Cargar las tablas desde la base de datos
query_production = "SELECT * FROM production;"
query_content = "SELECT * FROM content;"

df_production = pd.read_sql(query_production, conn)
df_content = pd.read_sql(query_content, conn)

# Cerrar la conexión
close_connection(conn, cur)

# Concatenar las tablas oscar, production y content
df_concat = pd.concat([df_production, df_content], axis=1)

Pasamos a cargar la base csv

In [4]:
df_csv = pd.read_csv('oscar.csv')

Concatenamos ambos dataframes para la base de datos final

In [7]:
df_netflix_oscar = pd.concat([df_concat, df_csv], axis=1)

In [8]:
df_netflix_oscar.head()

,id_production,id_content,title_production,genre,premiere,runtime,imdb_score,language,id_content,type,...,based_on,starring,distributed_by,budget,budget_x_million,box_office,box_office_x_million,imdb,metascore,rotten_tomatoes
0,1.0,1.0,Dick Johnson Is Dead,Documentary,"October 2, 2020",90.0,7.5,English,1,Movie,...,NaN,['George MacKay'. 'Dean-Charles Chapman'. 'Mar...,['Universal Pictures (Worldwide)'. 'Entertainm...,$95 million,95.0,$384.9 million,384.9,8.3,78.0,89.0
1,2.0,1093.0,Dolly Parton: A MusiCares Tribute,Documentary,"April 7, 2021",55.0,6.5,English,2,TV Show,...,NaN,['George MacKay'. 'Dean-Charles Chapman'. 'Mar...,['Universal Pictures (Worldwide)'. 'Entertainm...,$95 million,95.0,$384.9 million,384.9,8.3,78.0,89.0
2,3.0,1071.0,Why Did You Kill Me?,Documentary,"April 14, 2021",83.0,5.6,English,3,TV Show,...,NaN,['George MacKay'. 'Dean-Charles Chapman'. 'Mar...,['Universal Pictures (Worldwide)'. 'Entertainm...,$95 million,95.0,$384.9 million,384.9,8.3,78.0,89.0
3,4.0,142.0,Extraction,Action,"April 24, 2020",117.0,6.7,English,4,TV Show,...,NaN,['George MacKay'. 'Dean-Charles Chapman'. 'Mar...,['Universal Pictures (Worldwide)'. 'Entertainm...,$95 million,95.0,$384.9 million,384.9,8.3,78.0,89.0
4,5.0,3988.0,Antoine Griezmann: The Making of a Legend,Documentary,"March 21, 2019",60.0,6.5,French,5,TV Show,...,NaN,['George MacKay'. 'Dean-Charles Chapman'. 'Mar...,['Universal Pictures (Worldwide)'. 'Entertainm...,$95 million,95.0,$384.9 million,384.9,8.3,78.0,89.0


In [15]:
df_netflix_oscar.columns

Index(['id_production', 'id_content', 'title_production', 'genre', 'premiere',
       'runtime', 'imdb_score', 'language', 'id_content', 'type',
       'title_content', 'director', 'cast', 'country', 'date_added',
       'release_year', 'rating', 'duration', 'listed_in', 'description',
       'id_oscar', 'id_inf', 'id_res', 'id_content', 'title_oscar',
       'year_ceremony', 'category', 'name', 'winner', 'directed_by',
       'based_on', 'starring', 'distributed_by', 'budget', 'budget_x_million',
       'box_office', 'box_office_x_million', 'imdb', 'metascore',
       'rotten_tomatoes'],
      dtype='object')

In [16]:
# Eliminar columnas duplicadas
df_final = df_netflix_oscar.loc[:, ~df_netflix_oscar.columns.duplicated()]

Ahora que ya tenemos el dataframe final, puedo convertir para db, asi pasa a formar parte de mi base de datos. En este caso podria por ejemplo usar un administrador de base de datos y realizar consultas SQL si necesario. O bien, la opcion que elegimos para este proyecto, podemos pasar a trabajar el dataframe resultante haciendo su limpieza y preparacion para una posterior analisis

In [17]:
# Conectar a la base de datos
conn, cur = get_connection()

# Guardar el DataFrame en la base de datos
df_final.to_sql('oscar', conn, index=False, if_exists='replace')

# Cerrar la conexión
close_connection(conn, cur)

Averiguo que mi base de datos está completa, incluyendo las tablas que usamos para trabajar 'oscar', 'production' y 'content'

In [18]:
# Conectar a la base de datos
conn, cur = get_connection()

# Crear un cursor para ejecutar consultas SQL
cursor = conn.cursor()

# Obtener la lista de tablas en la base de datos
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Obtener los resultados de la consulta
tablas = cursor.fetchall()

# Cerrar el cursor y la conexión
cursor.close()
conn.close()

# Mostrar las tablas presentes en la base de datos
print("Tablas en la base de datos:")
for tabla in tablas:
    print(tabla[0])

Tablas en la base de datos:
sqlite_sequence
content
production
InformacionTabla
nombre_de_tabla
oscar
